# Models from scratch

We are gonna run our scratch models here, hypertune them, evaluate them and save the models.

In [1]:
# importing essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os 
import joblib

from sklearn.model_selection import RandomizedSearchCV   

# importing custom modules
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from src.data.load_data import load_data

from src.models.MLP_scratch import MLP as MLP_scratch, Trainer as Trainer_scratch
from src.models.tuning import run_random_search
# from src.utils.optimiziers import Adam, SGD, SGDMomentum, Adagrad
# from src.utils.losses import MAE, MSE, Huber, LogCosh
from src.visualization.model_plots import prediction_distribution, parity_plot, residual_plot,  plot_learning_curve




Loading in the `X` features and the specified `y` targets

In [2]:
target = "log_ClaimRate"

(X_train, y_train), (X_test, y_test)= load_data(raw=False, target=target)

## Decision Tree from Scratch

## Neural Network from Scratch

We make an instance of our Neural Network class.

In [3]:
MLP = Trainer_scratch(input_dim=X_train.shape[1], 
                      hidden_sizes=[32, 64],
                      model=MLP_scratch,
                      optimizer="adam",
                      loss_fn="mse",
                      epochs=100
                      )

we also need to define our parameter distributions for RandomizedGridSearchCV.

In [4]:
param_dist_MLP = {
    "hidden_sizes": [
        [32, 64], 
        [64, 128], 
        [128, 256],
        [32, 64, 128],
        [64, 128, 256],
        [8, 16, 32, 64],
        [16, 32, 64, 128],
    ],
    "optimizer": ["sgd", "sgd_momentum", "adam", "adagrad"],
    "loss_fn": ["mse", "mae", "huber", "logcosh"],
    "batch_size": [32, 64, 128, 256],
}


In [6]:
search = run_random_search(MLP, param_dist_MLP, X_train, y_train, n_iter=10, n_jobs=-1)

TypeError: Trainer.__init__() missing 4 required positional arguments: 'input_dim', 'hidden_sizes', 'optimizer', and 'loss_fn'

We have to choose the best model from this.

In [ ]:
best_model = search.best_estimator_


And we will also have to run it on the whole dataset(?)

Let's see how it did:

In [ ]:
y_train_pred = best_model.predict(X_train)
residual_plot(y_true=y_train, y_pred=y_train_pred)

In [ ]:
prediction_distribution(y_true=y_train, y_pred=y_train_pred)

In [ ]:
parity_plot(y_pred=y_train_pred, y_true=y_train)

In [ ]:
plot_learning_curve(best_model, X_train, y_train)

### Evaluation 

Let's use our model on the test data.

In [ ]:
y_test_pred = best_model.predict(X_test)

Plots:


In [ ]:
residual_plot(y_true=y_test, y_pred=y_test_pred)

In [ ]:
prediction_distribution(y_true=y_test, y_pred=y_test_pred)

In [ ]:
parity_plot(y_pred=y_test_pred, y_true=y_test)

### Saving the model

In [ ]:
joblib.dump(best_model, "../models/best_mlp_scratch_model.joblib")